want to use commonplayerinfo


In [10]:
from nba_api.stats.endpoints import commonplayerinfo
import pandas as pd
import time
import random
from requests.exceptions import ReadTimeout, ConnectionError
from http.client import RemoteDisconnected

# Read the CSV file with player IDs
player_ids = pd.read_csv('../data/players_2022_2023.csv')

# Create an empty list to store the data
data = []

# Function to get player info with retry mechanism
def get_player_info(player_id, retries=5, initial_delay=5):
    delay = initial_delay
    for attempt in range(retries):
        try:
            return commonplayerinfo.CommonPlayerInfo(player_id=player_id, timeout=60).get_data_frames()[0]
        except (ReadTimeout, ConnectionError, RemoteDisconnected) as e:
            if attempt < retries - 1:
                print(f"Error for player ID {player_id}: {e}. Retrying ({attempt + 1}/{retries})...")
                time.sleep(delay)
                delay *= 2  # Exponential backoff
                continue
            else:
                print(f"Failed to retrieve data for player ID {player_id} after {retries} attempts.")
                return None

# Loop through each player ID
for player_id in player_ids['PERSON_ID']:
    player_info = get_player_info(player_id)
    if player_info is not None:
        data.append(player_info)
    # Introduce a random delay to avoid hitting rate limits
    time.sleep(random.uniform(1, 3))

# Save the data to a CSV file if there is any data
if data:
    output_data = pd.concat(data)
    output_data.to_csv('output_data.csv', index=False)
    print(output_data)
else:
    print("No data was retrieved.")


Error for player ID 1628389: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=60). Retrying (1/5)...
    PERSON_ID FIRST_NAME  LAST_NAME DISPLAY_FIRST_LAST  \
0     1630173   Precious    Achiuwa   Precious Achiuwa   
0      203500     Steven      Adams       Steven Adams   
0     1628389        Bam    Adebayo        Bam Adebayo   
0     1630534      Ochai     Agbaji       Ochai Agbaji   
0     1630583      Santi     Aldama       Santi Aldama   
..        ...        ...        ...                ...   
0      201152   Thaddeus      Young     Thaddeus Young   
0     1629027       Trae      Young         Trae Young   
0     1630209       Omer  Yurtseven     Omer Yurtseven   
0      203469       Cody     Zeller        Cody Zeller   
0     1627826      Ivica      Zubac        Ivica Zubac   

   DISPLAY_LAST_COMMA_FIRST DISPLAY_FI_LAST       PLAYER_SLUG  \
0         Achiuwa, Precious      P. Achiuwa  precious-achiuwa   
0             Adams, Steven        S. 